In [1]:
# import spacy
# from spacy.matcher import Matcher
# from spacy.matcher import PhraseMatcher
# from spacy import displacy
import re,string

import docx
import os
import emoji

import pandas as pd
import jieba
import jieba.posseg as pseg
from pyltp import SentenceSplitter,Segmentor,Postagger,NamedEntityRecognizer,Parser,SementicRoleLabeller


In [2]:
filePath = './data/solution/'

In [3]:
# 文件操作
# 读取docx文件
def readDocx(filePath):
    fullText=[]
    # fullText=""
    doc = docx.Document(filePath)
    for p in doc.paragraphs:
        fullText.append(p.text)
        # fullText = fullText+(p.text)
    return fullText

# 读取所有文件
def findAllFile(base):
    for root,dirs,files in os.walk(base):
        for f in files:
            yield f

# 写入csv中
def writeCSV(data_ls):
    df=pd.DataFrame(data_ls)
    df.to_csv(csvPath,mode='a',index=False)

# 写入docx
def writeDocx(text,filePath):
    doc = docx.Document()
    p = doc.add_paragraph(text)
    doc.save(filePath)


# filePath_ls是solution文件夹下所有文件的文件名
def getAllFilePath(filePath_ls):
    for file in findAllFile(filePath):
        filePath_ls.append(file)
    # filePath_ls.sort(key=lambda x:int(x.split('.')[:-1][0]))

In [4]:
# 数据预处理

# 1、将表情包转为符号表示
# 2、删除所有的英文，下划线，数字

def remove_special_characters(text):
    char_pattern = re.compile(r'[\n\r\t]')

    return char_pattern.sub('',text)

def emoji2code(text):
    return emoji.demojize(text)


# 3.删除表情符号
def remove_emoji(text):
    res = emoji.demojize(text)
    try:  
        co = re.compile(u'['u'\U0001F300-\U0001F64F' u'\U0001F680-\U0001F6FF'u'\u2600-\u2B55]+')  
    except re.error:  
        co = re.compile(u'('u'\ud83c[\udf00-\udfff]|'u'\ud83d[\udc00-\ude4f\ude80-\udeff]|'u'[\u2600-\u2B55])+')  
    return co.sub('', res)


# 删除markdown语法
def remove_markdown(text):
    markdown_pattern = re.compile(r'```.*?```|:.*?:|!\[.*?\)|[#*$\\\|]|<!.*?>|\[.*?\)')
    return markdown_pattern.sub('',text)

# 删除标点符号
def remove_punct(text):
    punct = "-：`+=(∩){},/;&.（）>!%\"'<\[\]:?“”、！①②③④⑤？—「」^~【】@Σ∣"
    return re.sub(r"[%s]+" %punct, "",text)

# 删除字母数字下划线
def remove_num_alpha(text):
    pattern = re.compile(r'[A-Za-z0-9_]')
    return pattern.sub('',text)

# 删除空格
def remove_whiteSpace(text):
    return text.replace(" ","")

#创建停用词表
# def remove_stopwords(filepath,text):
# 	stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]
# 	for words in stopwords:
#         if not 
#     # return stopwords



# 分句，也就是将一片文本分割为独立的句子
def sentence_splitter(sentence):
	sents = SentenceSplitter.split(sentence)  # 分句
	return sents
	# print(sents)
	# print('\n'.join(sents))


def complete_noise(data):
    new_data = remove_special_characters(data)
    new_data = emoji2code(new_data)
    new_data = remove_markdown(new_data)
    
    new_data = remove_num_alpha(new_data)
    new_data = remove_punct(new_data)
    new_data = remove_whiteSpace(new_data)
    return new_data

def create_text(list):
    result_ls = []
    for item in list:
        result_ls.append(str(complete_noise(item)))
    
    result_str = ''.join(result_ls)
    return result_str


In [5]:
LTP_DATA_DIR='D:\LTPmodel\ltp_data_v3.4.0'
cws_model_path = os.path.join(LTP_DATA_DIR, 'cws.model')  # 分词模型路径，模型名称为`cws.model`
pos_model_path = os.path.join(LTP_DATA_DIR, 'pos.model')  # 词性标注模型路径，模型名称为`pos.model`
ner_model_path = os.path.join(LTP_DATA_DIR, 'ner.model')  # 命名实体识别模型路径，模型名称为`ner.model`
par_model_path = os.path.join(LTP_DATA_DIR, 'parser.model')	 # 依存句法分析模型路径，模型名称为`parser.model`
srl_model_path = os.path.join(LTP_DATA_DIR, 'srl')	# 语义角色标注模型目录路径，模型目录为`srl`。注意该模型路径是一个目录，而不是一个文件。

In [6]:
def segmentor(sentence):
	segmentor = Segmentor(cws_model_path,'D:\zoe\实验\meProject\Algorithm_KnowledgeGraph\data\lexicon.txt')	 # 初始化实例
	# segmentor.load(cws_model_path)	# 加载模型
	#segmentor.load_with_lexicon('cws_model_path', 'D:\pyprojects\LTP\ltp_data\dict.txt') #加载模型	  使用用户自定义字典的高级分词
	words = segmentor.segment(sentence)	 # 分词
	# 默认可以这样输出
	
	# 可以转换成List 输出
	# words_list = list(words)
	segmentor.release()	 # 释放模型
	return words

In [ ]:
list = readDocx('./data/solution/2.两数相加.docx')
data=create_text(list)
print(data)

In [ ]:
# 分句
sent = sentence_splitter(data)
for s in sent:
    print(s,'\n')

In [18]:
# 分词
seg = segmentor(sent[7])
print(seg)

['此外', '，', '如果', '链表', '遍历', '结束', '后', '，', '有', '，', '还', '需要', '在', '答案', '链表', '的', '后面', '附加', '一个', '节点', '，', '节点', '的', '值', '为', '。']


In [15]:
#创建停用词表
def remove_stopwords_punct(text,path):
	result_ls = []
	stopwords = [line.strip() for line in open(path, 'r', encoding='utf-8').readlines()]
	for word in text:
		if word not in stopwords:
			result_ls.append(str(word))
	result_str = ''.join(result_ls)
	return result_str


In [16]:
stoppath = 'D:\zoe\实验\meProject\Algorithm_KnowledgeGraph\stopwords\ltp_stopwords.txt'

In [19]:
print(remove_stopwords_punct(seg,stoppath))

链表遍历结束后还需要答案链表后面附加节点节点值


In [ ]:
for s in sent:
    seg = segmentor(s)
    seg = remove_stopwords_punct(seg,stoppath)


In [163]:
filePath_ls=[]
getAllFilePath(filePath_ls)
for index in range(1,2):
    path = filePath+filePath_ls[index]
    list=readDocx(path)
    doc = docx.Document()
    data=create_text(list)
    sent = sentence_splitter(data)
    for s in sent:
        seg = segmentor(s)
        seg = remove_stopwords_punct(seg,stoppath)
        p = doc.add_paragraph(seg)
    doc.save(path)
    print(index,'write successfully!')
# for file in filePath_ls:
#     path=filePath+file
#     text=readDocx(path)
#     ls=[]
#     for item in text:
#         item=remove_special_characters(item)
#         item=remove_stop_words_and_punct(item)
#         matche(ls,item,dic_ls)
#     ls=remove_duplicates(ls)
#     data_ls.append(ls)
        
#     print(file+"文件匹配完毕！")

1 write successfully!
